# Extraction of Representations from Deep Neural Networks


One of the most basic and useful operations on a deep network is the extraction
of representations in its hidden layers.

The reason is that representations are in themselves a remarkable object of
investigation. 
Representations are the encoding of the data at a given layer, and their categorical
content can be decoded.
The performance of this decoding can tell us how explicitly the categorical information is contained in the representation.
In this exercise we will extract representations in hidden layers when an input is given. The way to do that in Keras is not exactly transparent, but we will show it.


<img src="../NotebooksFigures/tikz41.png" alt="drawing" width="800" >



In [29]:
# imports

import os
from os import listdir 
from os.path import isfile, join
import numpy as np
from matplotlib import pyplot as plt
import h5py

from keras.datasets import mnist
from keras import layers
from keras.models import Model, load_model
from keras import optimizers
from keras import losses
from keras import callbacks
from keras.utils.np_utils import to_categorical

# Load data

In [30]:
# this is the same code used for the training sessions

(i_train, l_train), (i_test, l_test) = mnist.load_data()
i_train, i_test = i_train/255.0, i_test/255.0
i_train, i_test = i_train.astype(np.float32), i_test.astype(np.float32)
i_train, i_test = np.expand_dims(i_train, 3), np.expand_dims(i_test, 3)
l_train, l_test = to_categorical(l_train), to_categorical(l_test)

i_validate = i_train[50000:, :, :, :]
i_train = i_train[0:50000, :, :, :]
l_validate = l_train[50000:, :]
l_train = l_train[0:50000, :]

# Model construction

We build a multi-layer perceptron, similar to the one in the figure at the beginning of the notebook. We build the network in Keras but in a more 'pythonic'
way, with respect to the training session, defining a network class and its methods.

In [31]:
minibatch_size = 64
n_epochs = 10
learn_rate = 0.1
history = {}
tb_params = {
    'write_images': True, 'histogram_freq': 5, 
    'write_grads': True, 'write_graph': False
            }
activation_fcn = 'sigmoid'

In [32]:
img = layers.Input(shape=(28, 28, 1,), name='images')
x = layers.Flatten()(img)
x = layers.Dense(100, activation=activation_fcn, name='hidden1')(x)
x = layers.Dense(100, activation=activation_fcn, name='hidden2')(x)
x = layers.Dense(100, activation=activation_fcn, name='hidden3')(x)
out = layers.Dense(10, activation='softmax', name='output')(x)
model = Model(img, out)

In [33]:
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
images (InputLayer)          (None, 28, 28, 1)         0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 784)               0         
_________________________________________________________________
hidden1 (Dense)              (None, 100)               78500     
_________________________________________________________________
hidden2 (Dense)              (None, 100)               10100     
_________________________________________________________________
hidden3 (Dense)              (None, 100)               10100     
_________________________________________________________________
output (Dense)               (None, 10)                1010      
Total params: 99,710
Trainable params: 99,710
Non-trainable params: 0
_________________________________________________________________
None


# Training

In [6]:
opt = optimizers.SGD(lr=learn_rate)
model.compile(optimizer=opt, loss=losses.categorical_crossentropy, metrics=['accuracy'])
tb = callbacks.TensorBoard(log_dir=join('tb_data', 'fully_connected'), **tb_params)
h = model.fit(
    i_train, l_train, validation_data=(i_validate, l_validate), epochs=n_epochs,
    batch_size=minibatch_size, callbacks=[tb]
    )
history['Fully Connected'] = h.history
history['Fully Connected']['epoch'] = h.epoch

Train on 50000 samples, validate on 10000 samples
Epoch 1/10
50000/50000 [==============================] - 1s 23us/step - loss: 2.2570 - acc: 0.1749 - val_loss: 2.0579 - val_acc: 0.2246
Epoch 2/10
50000/50000 [==============================] - 1s 21us/step - loss: 1.3081 - acc: 0.5765 - val_loss: 0.7929 - val_acc: 0.7429
Epoch 3/10
50000/50000 [==============================] - 1s 21us/step - loss: 0.6937 - acc: 0.7907 - val_loss: 0.5519 - val_acc: 0.8509
Epoch 4/10
50000/50000 [==============================] - 1s 21us/step - loss: 0.5024 - acc: 0.8597 - val_loss: 0.4284 - val_acc: 0.8771
Epoch 5/10
50000/50000 [==============================] - 1s 21us/step - loss: 0.4046 - acc: 0.8842 - val_loss: 0.3521 - val_acc: 0.8981
Epoch 6/10
50000/50000 [==============================] - 1s 21us/step - loss: 0.3515 - acc: 0.8994 - val_loss: 0.3284 - val_acc: 0.9057
Epoch 7/10
50000/50000 [==============================] - 1s 21us/step - loss: 0.3140 - acc: 0.9112 - val_loss: 0.2764 - val_acc

# Representations extraction

In [18]:
input_shape = i_train.shape[1:]
print(input_shape)

(28, 28, 1)


In [22]:
nsamples = 1000
test = i_test[np.random.permutation(i_test.shape[0])[0:nsamples],:,:,: ]

In [23]:
print(test.shape)

(1000, 28, 28, 1)


In [24]:
from keras import backend as K

inp = model.input                                           # input placeholder
outputs = [layer.output for layer in model.layers]          # all layer outputs
functor = K.function([inp, K.learning_phase()], outputs )   # evaluation function


layer_outs = functor([test, 1.])

In [27]:
?K.function

Signature: K.function(inputs, outputs, updates=None, **kwargs)
Docstring:
Instantiates a Keras function.

# Arguments
    inputs: List of placeholder tensors.
    outputs: List of output tensors.
    updates: List of update ops.
    **kwargs: Passed to `tf.Session.run`.

# Returns
    Output values as Numpy arrays.

# Raises
    ValueError: if invalid kwargs are passed in.
File:      ~/.local/envs/tf/lib/python3.5/site-packages/keras/backend/tensorflow_backend.py
Type:      function


In [28]:
?K.learning_phase

Signature: K.learning_phase()
Docstring:
Returns the learning phase flag.

The learning phase flag is a bool tensor (0 = test, 1 = train)
to be passed as input to any Keras function
that uses a different behavior at train time and test time.

# Returns
    Learning phase (scalar integer tensor or Python integer).
File:      ~/.local/envs/tf/lib/python3.5/site-packages/keras/backend/tensorflow_backend.py
Type:      function


In [25]:
_ = [print(l.shape) for l in layer_outs]

(1000, 28, 28, 1)
(1000, 784)
(1000, 100)
(1000, 100)
(1000, 100)
(1000, 10)


array([[0.94341296, 0.68029934, 0.18934679, ..., 0.5720547 , 0.8121603 ,
        0.6126356 ],
       [0.7269921 , 0.5324204 , 0.30694357, ..., 0.7420718 , 0.7657363 ,
        0.4069131 ],
       [0.13606167, 0.62172663, 0.19126253, ..., 0.89357895, 0.25269952,
        0.742206  ],
       ...,
       [0.98225236, 0.7430345 , 0.05686154, ..., 0.99125755, 0.48475155,
        0.2932695 ],
       [0.75308293, 0.554284  , 0.23079765, ..., 0.8703133 , 0.5700442 ,
        0.9495621 ],
       [0.82203573, 0.61570245, 0.03745488, ..., 0.8826881 , 0.88168645,
        0.95447797]], dtype=float32)